In [20]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print('device:', device)

plt.ion()   # interactive mode

device: mps


In [21]:
class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as member variables.
        """
        super(TwoLayerNet, self).__init__()
        # the weight and bias of linear1 will be initialized 
        # you can access them by self.linear1.weight and self.linear1.bias
        self.linear1 = nn.Linear(D_in, H, device) # this will create weight, bias for linear1
        self.linear2 = nn.Linear(H, D_out, device) # this will create weight, bias for linear2
        self.sigmoid = nn.Sigmoid() # Sigmoid activation for binary classification

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return a Tensor of output data.
        We can use Modules defined in the constructor as well as arbitrary operators on Tensors.
        """
        x.to(device) # move the data to the device (cpu or gpu)
        h_relu = F.relu(self.linear1(x))
        y_pred = F.sigmoid(self.linear2(h_relu), dim=1)
        return y_pred

In [39]:
# use sklearn.preprocessing.MinMaxScaler to transform features by scaling each feature to a given range.
from sklearn.preprocessing import MinMaxScaler

def TensorDataset(root_dir, train):
    scaler = MinMaxScaler()
    titanic = pd.read_csv(root_dir)
    titanic.fillna(0, inplace=True) # filled the missing values with 0 NaN 
 
    # only need "Pclass", "Age", "SibSp", "Parch", „Fare“, „Sex“, „Embarked“
    titanic = titanic[["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex", "Embarked", "Survived"]]
    # if Sex is "male", replace it with 1, otherwise replace it with 0
    titanic["Sex"] = titanic["Sex"].apply(lambda x: (x == 'male' and 1) or 0)
    # if Embarked is "S", replace it with 0, if "C", replace it with 1, if "Q", replace it with 2
    titanic["Embarked"] = titanic["Embarked"].apply(lambda x: (x == 'S' and 0) or (x == 'C' and 1) or 2)
    titanic[["Pclass", "Age", "SibSp", "Parch", "Fare"]] = scaler.fit_transform(titanic[["Pclass", "Age", "SibSp", "Parch", "Fare"]])

    # split the data into train and validation data (80% and 20%).
    if train:
        # return 80% of the data
        return titanic.sample(frac=0.8, random_state=42)  # Use a random_state for reproducibility
    else: 
        # return 20% of the data
        return titanic.sample(frac=0.2, random_state=42)  # Use the same random_state for consistency


In [40]:
train_dataset = TensorDataset('./data/titanic.csv', train=True)
val_dataset = TensorDataset('./data/titanic.csv', train=False)
print('train_dataset len:', len(train_dataset))
print('val_dataset len:', len(val_dataset))


train_dataset len: 713
val_dataset len: 178


In [41]:
train_dataset

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked,Survived
709,1.0,0.00000,0.125,0.166667,0.029758,1,1,1
439,0.5,0.38750,0.000,0.000000,0.020495,1,2,0
840,1.0,0.25000,0.000,0.000000,0.015469,1,2,0
720,0.5,0.07500,0.000,0.166667,0.064412,0,2,1
39,1.0,0.17500,0.125,0.000000,0.021942,0,1,1
...,...,...,...,...,...,...,...,...
639,1.0,0.00000,0.125,0.000000,0.031425,1,2,0
878,1.0,0.00000,0.000,0.000000,0.015412,1,2,0
824,1.0,0.02500,0.500,0.166667,0.077465,1,2,0
803,1.0,0.00525,0.000,0.166667,0.016623,1,1,1


In [32]:
import torch.optim as optim

# N is batch size; 
# D_in is input dimension; 
# H is hidden dimension (Only one hidden layer, but containing 3 neurons.); 
# D_out is output dimension
N, D_in, H, D_out = 16, 7, 3, 2
lr = 0.001


model = TwoLayerNet(D_in, H, D_out)  # H=3 for one hidden layer with 3 neurons
optimizer = optim.Adam(model.parameters(), lr)  # You can adjust the learning rate as needed
criterion = nn.BCELoss() # Define the loss function as Binary Cross-Entropy Loss
num_epochs = 1 # You can adjust the number of epochs as needed

In [33]:
train_dataset.shape[1]

8

In [ ]:
for epoch in range(num_epochs):
    model.train
    